## PHYS 249 Assignment \#3:  Integration and Lookup Tables 

(Based on Newman question 5.17 - see page 205 for details)

**1** Consider the gamma function $\Gamma(a)$, which is defined by the integral
$$\Gamma(a) = \int^\infty_0 x^{a-1}e^{-x}dx\,.$$
There is no closed form expression for this integral, so we have to evluate it numerically.

Before we calculate the integral, we have to make two transformations to the integrand. First, to map the infinite range onto a finite one, in a way that places the peak of the function at the middle of the interval, we can use the transformation

$$z = \frac{x}{a-1+x}$$

or equivalently

$$x = (a-1)\frac{z}{1-z};\ \ \ \ dx = (a-1)\frac{dz}{(1-z)^2}$$

Second, to avoid multiplying very large numbers by very small numbers in the integrand, it is also convenient to write it as a single exponential

$$x^{a-1}e^{-x} = e^{(a-1)\ln x}e^{-x} = \exp[(a-1)\ln x - x]\,.$$

**The problem with these two transformations is that they only work for $a > 1$. Thus a full solution to calculating $\Gamma(a)$ involves treating the cases $a \le 1$ and $a > 1$ separately. Here, to keep things simple, I'll ask you just to consider the case $a > 1$.** 


**a)** Using these two transformations, rewrite the integrand as a well-behaved function of $z$ on the interval 0 to 1 (assuming $a> 1$). (Use the latex math symbols available in Jupyter notebooks to write out the mathematical expression in a text box below.)



$$x^{a-1}e^{-x} = e^{(a-1)\ln x}e^{-x} = \exp[(a-1)\ln x - x]\,.$$
$$\Gamma(a) = \int^\infty_0 x^{a-1}e^{-x}dx\,.$$ 
$$ = \int^\infty_0 e^{(a-1)\ln x - x}dx$$
$$ (a-1)\ln x -x => (a-1)\ln(\frac{z*(a-1)}{1-z}) - (a-1)\frac{z}{1-z} $$
$$ => (a - 1) * (\ln(\frac{z*(a-1)}{1-z}) - \frac{z}{1-z}) $$ 
$$ \Gamma(a) = \int_{0}^{1} e^{(a - 1) * (\ln(\frac{z*(a-1)}{1-z}) - \frac{z}{1-z})}dx\, $$
$$ \Gamma(a) = \int_{0}^{1} e^{(a - 1) * (\ln(\frac{z*(a-1)}{1-z}) - \frac{z}{1-z})}\frac{a-1}{(1-z)^2}dz\, $$


**b)** Then write Python code to evaluate $\Gamma(a)$ for $a > 1$ by integrating this function, using any method you have seen (i.e. a Riemann sum, the trapzoid rule, or Simpson's rule, but **not** another method or prepackaged function). You can skip the end points of the interval if the integrand is badly behaved there. You can test your code by calculating $\Gamma(1.5)$, which is known to be equal to $\frac{1}{2}\sqrt{\pi}$. Use enough subintervals in your calculation to get a relative accuracy of $\sim 10^{-5}$.

In [3]:
from math import log, exp, pi

def integrate(a, b, n, fun):
    
    deltax = (b-a)/n
    a = a + deltax
    b = b - deltax
    n -= 2
    sum = 0
    evensum = 0
    oddsum = 0
    for i in range(1, n, 2):
        # print(i*deltax)
        evensum += fun(a + i*deltax)

    evensum *= 4
    for i in range(2, n, 2):
        # print(i*deltax)
        oddsum += fun(a + i*deltax)

    # print('check')
    oddsum *= 2
    sum += evensum + oddsum + fun(a) + fun(b)
    # print('check2')
    sum *= (1/3)*deltax
    return sum


power = lambda z, a : (a - 1) * (log (z * (a - 1)/(1 - z)) - z/(1 - z))

integrand = lambda a : lambda z : exp(power(z, a)) * (a-1)/((1-z)*(1-z))
# print(integrand(1.5)(0.5))

# print(integrate(0, 1, 100, lambda x : x*x))

real = 0.5 * (pi)**(0.5)
temp = integrate(0, 1, 1000, integrand(1.5))
print(temp)
print(abs(real - temp)/real)





0.8862194489500245
8.436329927182712e-06


**c)** Evaluate $\Gamma(a_i)$ for $a_i = 1.1-10$, in steps of 0.1 and save the results in an array. 

Use these values to implement a **lookup table**. This is a Python function that for a given value $a_{new}$ on the interval $[1.1,10]$, estimates the value of $\Gamma(a_{new})$ by using **linear interpolation** on the previously calculated $\Gamma(a_i)$.

To do this, you will have to put in a loop with a logic test to find the two values $a_i, a_{i+1}$ that bracket $a_{new}$. Then you can use the linear interpolation formula from slides 6-7 of unit 4:

$$\Gamma(a_{new}) \sim \frac{(a_{i+1} - a_{new})}{(a_{i+1} - a_i)}\Gamma(a_i) + \frac{(a_{new} - a_i)}{(a_{i+1} - a_i)}\Gamma(a_{i+1})$$



In [4]:
gamma = {}
for i in range(1, 91, 1):
    gamma[1 + 0.1 * i] = integrate(0, 1, 1000, integrand(1 + 0.1 * i))



def get_new(a):
    a_n = a
    a *= 10
    a = int(a)
    a_i = a / 10
    a_i2 = a_i + 0.1
    # print(a_i, a_i2)
    res = (a_i2 - a_n) * gamma[a_i]/(0.1) + (a_n - a_i) * gamma[a_i2]/(0.1)
    return res



print(get_new(1.55))



0.8898652051743738


**d)** Estimate the relative accuracy of your lookup table with respect to the full integration, by comparing $\Gamma(1.55)$ estimated from the lookup table to $\Gamma(1.55)$ calculated by your code from part **b)**.

In [7]:
temp = integrate(0, 1, 1000, integrand(1.55))
print("Calculated: {}".format(temp))
lookup = get_new(1.55)
print("Lookup table: {}".format(lookup))
print("Relative accuracy: {}".format(abs((temp-lookup)/temp)))

Calculated: 0.8888626122664531
Lookup table: 0.8898652051743738
Relative accuracy: 0.001127950364977396
